In [1]:
import sys
import os

# Agregar rutas del proyecto
sys.path.append(os.path.abspath(".."))  # raíz del proyecto
sys.path.append(os.path.abspath("../src"))  # carpeta src

# Verificar rutas
print("Rutas configuradas:")
for path in sys.path[-3:]:
    print(f"  - {path}")

Rutas configuradas:
  - /home/arturo/venv/lib/python3.12/site-packages
  - /home/arturo/project_gutenberg
  - /home/arturo/project_gutenberg/src


In [2]:
import nltk

print("Descargando recursos de NLTK...")
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("✓ NLTK configurado")

Descargando recursos de NLTK...
✓ NLTK configurado


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProyectoFinal_RecomendacionLibros") \
    .master("local[*]") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Reducir logs
spark.sparkContext.setLogLevel("WARN")

print("✓ Spark inicializado")
print(f"  Version: {spark.version}")
print(f"  Master: {spark.sparkContext.master}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/06 01:13:43 WARN Utils: Your hostname, arturo-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/12/06 01:13:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/06 01:13:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Spark inicializado
  Version: 4.0.1
  Master: local[*]


In [4]:
from src.utils import (
    read_txt,
    strip_gutenberg_headers,
    preprocess_text,
    load_all_books
)

print("✓ Utilidades importadas")

✓ Utilidades importadas


In [5]:
data_dir = "../data"
books = load_all_books(data_dir, max_books=100)

print(f"\nResumen de carga:")
print(f"   Total de libros: {len(books)}")
if books:
    ejemplo = books[0]
    print(f"   Ejemplo - ID: {ejemplo[0]}, Tokens: {len(ejemplo[3])}")


📚 Cargando 97 libros desde ../data/
  ✓ Procesados 10/97 libros
  ✓ Procesados 20/97 libros
  ✓ Procesados 30/97 libros
  ✓ Procesados 40/97 libros
  ✓ Procesados 50/97 libros
  ✓ Procesados 60/97 libros
  ✓ Procesados 70/97 libros
  ✓ Procesados 80/97 libros
  ✓ Procesados 90/97 libros
✅ Total de libros cargados exitosamente: 97


Resumen de carga:
   Total de libros: 97
   Ejemplo - ID: 100, Tokens: 498519


In [6]:
from pyspark.sql.types import ArrayType, StringType

print("Creando DataFrame de Spark...")

df = spark.createDataFrame(
    [(b[0], b[1], b[2], b[3]) for b in books],
    schema=["book_id", "title", "text", "tokens"]
)

print(f"✓ DataFrame creado con {df.count()} documentos\n")
df.select("book_id", "title").show(5, truncate=False)


Creando DataFrame de Spark...


25/12/06 01:14:43 WARN TaskSetManager: Stage 0 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ DataFrame creado con 97 documentos



25/12/06 01:15:48 WARN TaskSetManager: Stage 3 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+-------+--------+
|book_id|title   |
+-------+--------+
|100    |100.txt |
|103    |103.txt |
|105    |105.txt |
|1080   |1080.txt|
|11     |11.txt  |
+-------+--------+
only showing top 5 rows


In [7]:
from pyspark.sql.functions import col

df = df.withColumnRenamed("tokens", "tokens_clean")

print("✓ Tokens ya procesados (stopwords removidas con NLTK)")
print("\nEjemplo de tokens limpios:")
df.select("book_id", "tokens_clean").show(3, truncate=60)

✓ Tokens ya procesados (stopwords removidas con NLTK)

Ejemplo de tokens limpios:


25/12/06 01:16:40 WARN TaskSetManager: Stage 4 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+-------+------------------------------------------------------------+
|book_id|                                                tokens_clean|
+-------+------------------------------------------------------------+
|    100|[complete, works, william, shakespeare, william, shakespe...|
|    103|[illustration, around, world, eighty, days, jules, verne,...|
|    105|[persuasion, jane, austen, contents, chapter, chapter, ii...|
+-------+------------------------------------------------------------+
only showing top 3 rows


In [8]:
from pyspark.ml.feature import CountVectorizer

print("Creando vocabulario con CountVectorizer...")

VOCAB_SIZE = 2000
MIN_DF = 2  # Palabra debe aparecer en al menos 2 documentos

cv = CountVectorizer(
    inputCol="tokens_clean",
    outputCol="raw_features",
    vocabSize=VOCAB_SIZE,
    minDF=MIN_DF
)

cv_model = cv.fit(df)
df = cv_model.transform(df)

actual_vocab_size = len(cv_model.vocabulary)
print(f"✓ Vocabulario creado")
print(f"  Tamaño del vocabulario: {actual_vocab_size} palabras")
print(f"  Top 10 palabras: {cv_model.vocabulary[:10]}")

Creando vocabulario con CountVectorizer...


25/12/06 01:18:05 WARN TaskSetManager: Stage 5 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ Vocabulario creado
  Tamaño del vocabulario: 2000 palabras
  Top 10 palabras: ['said', 'one', 'would', 'man', 'de', 'could', 'que', 'like', 'time', 'well']


In [9]:
from pyspark.ml.feature import IDF

print("\nCalculando TF-IDF...")

idf = IDF(inputCol="raw_features", outputCol="tfidf")
idf_model = idf.fit(df)
df = idf_model.transform(df)

print("✓ TF-IDF calculado")


Calculando TF-IDF...


25/12/06 01:19:25 WARN TaskSetManager: Stage 9 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

✓ TF-IDF calculado


In [10]:
from pyspark.ml.feature import Normalizer

print("Normalizando vectores TF-IDF...")

normalizer = Normalizer(inputCol="tfidf", outputCol="tfidf_norm", p=2.0)
df = normalizer.transform(df)

print("✓ Vectores normalizados")
print("\nDataFrame final:")
df.select("book_id", "title", "tfidf_norm").show(3, truncate=60)

Normalizando vectores TF-IDF...
✓ Vectores normalizados

DataFrame final:


25/12/06 01:19:56 WARN TaskSetManager: Stage 10 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
[Stage 10:>                                                         (0 + 1) / 1]

+-------+-------+------------------------------------------------------------+
|book_id|  title|                                                  tfidf_norm|
+-------+-------+------------------------------------------------------------+
|    100|100.txt|(2000,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,...|
|    103|103.txt|(2000,[0,1,2,3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,2...|
|    105|105.txt|(2000,[0,1,2,3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,2...|
+-------+-------+------------------------------------------------------------+
only showing top 3 rows


In [11]:
import numpy as np

print("Creando matriz de similitud...")

# Recolectar datos
rows = df.select("book_id", "title", "tfidf_norm").collect()

# Preparar estructuras de datos
book_ids = [r["book_id"] for r in rows]
id_to_title = {r["book_id"]: r["title"] for r in rows}

# Convertir vectores a numpy
print("  → Convirtiendo vectores a numpy...")
vectors = np.array([r["tfidf_norm"].toArray() for r in rows])

# Calcular matriz de similitud (producto punto = cosine similarity)
print("  → Calculando similitudes...")
sim_matrix = np.dot(vectors, vectors.T)

# Crear mapeos
index_to_id = {i: book_ids[i] for i in range(len(book_ids))}
id_to_index = {book_ids[i]: i for i in range(len(book_ids))}

print(f"✓ Matriz de similitud creada: {sim_matrix.shape}")
print(f"  Rango de similitudes: [{sim_matrix.min():.4f}, {sim_matrix.max():.4f}]")


Creando matriz de similitud...


25/12/06 01:20:03 WARN TaskSetManager: Stage 11 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

  → Convirtiendo vectores a numpy...
  → Calculando similitudes...
✓ Matriz de similitud creada: (97, 97)
  Rango de similitudes: [0.0000, 1.0000]


In [12]:
def recomendar_libros(libro_id, N=5):
    libro_id = str(libro_id)  # <-- solución
    
    if libro_id not in id_to_index:
        raise ValueError(f"❌ Libro '{libro_id}' no encontrado")
    
    idx = id_to_index[libro_id]
    similarities = sim_matrix[idx]

    pairs = [
        (index_to_id[i], float(similarities[i]))
        for i in range(len(similarities))
        if i != idx
    ]

    pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
    
    results = [
        (bid, id_to_title[bid], score)
        for bid, score in pairs_sorted[:N]
    ]
    
    return results


In [13]:
recomendar_libros(100, N=5)

[('1112', '1112.txt', 0.7441848048758577),
 ('58', '58.txt', 0.7253547529834251),
 ('3296', '3296.txt', 0.7028455377826147),
 ('2265', '2265.txt', 0.652600381541398),
 ('1322', '1322.txt', 0.5827439799498586)]

In [14]:
from pyspark.sql.functions import col

def palabras_importantes(documento_id, M=10):
    """
    🎯 FUNCIÓN PRINCIPAL #2
    
    Regresa M palabras que describen un documento.
    
    Parámetros:
        documento_id: ID del documento (book_id)
        M: Número de palabras a retornar
    
    Retorna:
        Lista de tuplas (palabra, tf-idf_score)
    """
    # Buscar documento
    row = df.filter(col("book_id") == documento_id).select("tfidf").collect()
    
    if not row:
        raise ValueError(f"❌ Documento '{documento_id}' no encontrado")
    
    # Obtener vector TF-IDF
    tfidf_vector = row[0]["tfidf"]
    vocab = cv_model.vocabulary
    
    # Ordenar por score TF-IDF
    items = list(zip(tfidf_vector.indices, tfidf_vector.values))
    items_sorted = sorted(items, key=lambda x: x[1], reverse=True)
    
    # Convertir a palabras
    top_words = [
        (vocab[idx], float(val))
        for idx, val in items_sorted[:M]
    ]
    
    return top_words


print("✓ Función palabras_importantes() definida")

✓ Función palabras_importantes() definida


In [15]:
palabras_importantes(84, M=5)

25/12/06 01:20:37 WARN TaskSetManager: Stage 12 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

[('elizabeth', 129.00312038398658),
 ('feelings', 24.493857723823492),
 ('william', 21.192334148441287),
 ('misery', 19.36936799520666),
 ('henry', 18.151825972355446)]

In [18]:
print("\n" + "="*70)
print("🔍 EJEMPLO 1: SISTEMA DE RECOMENDACIÓN")
print("="*70)

# Escoger un libro de ejemplo (puedes cambiar el índice)
ejemplo_libro_id = "84"  # Primer libro

print(f"\n📖 Libro base:")
print(f"   ID: {ejemplo_libro_id}")
print(f"   Título: {id_to_title[ejemplo_libro_id]}")

print(f"\n🎯 Top 5 libros recomendados:")
print("-" * 70)

recomendaciones = recomendar_libros(ejemplo_libro_id, N=5)

for i, (bid, title, score) in enumerate(recomendaciones, 1):
    print(f"{i}. [{score:.4f}] {title}")
    print(f"   ID: {bid}\n")


🔍 EJEMPLO 1: SISTEMA DE RECOMENDACIÓN

📖 Libro base:
   ID: 84
   Título: 84.txt

🎯 Top 5 libros recomendados:
----------------------------------------------------------------------
1. [0.6763] 1342.txt
   ID: 1342

2. [0.4039] 2148.txt
   ID: 2148

3. [0.3877] 4085.txt
   ID: 4085

4. [0.3120] 205.txt
   ID: 205

5. [0.2673] 209.txt
   ID: 209



In [21]:
print("\n" + "="*70)
print("🎮 MODO INTERACTIVO")
print("="*70)

# Función helper para análisis rápido
def analizar_libro(libro_id, n_recomendaciones=5, m_palabras=10):
    """Función de ayuda para análisis rápido"""
    print(f"\n{'='*70}")
    print(f"📊 ANÁLISIS DEL LIBRO: {libro_id}")
    print(f"{'='*70}")
    
    print(f"\n📖 Información:")
    print(f"   Título: {id_to_title[libro_id]}")
    
    # Palabras características
    print(f"\n📝 Top {m_palabras} palabras características:")
    palabras = palabras_importantes(libro_id, M=m_palabras)
    for i, (palabra, score) in enumerate(palabras, 1):
        print(f"   {i:2}. {palabra:15} (TF-IDF: {score:.2f})")
    
    # Recomendaciones
    print(f"\n🎯 Top {n_recomendaciones} libros similares:")
    recs = recomendar_libros(libro_id, N=n_recomendaciones)
    for i, (bid, title, score) in enumerate(recs, 1):
        print(f"   {i}. [{score:.4f}] {title}")
    
    print("="*70)

# Ejemplo de uso:
print("\n💡 Usa la función analizar_libro() para explorar:")
print('   analizar_libro("1342", n_recomendaciones=5, m_palabras=10)')



🎮 MODO INTERACTIVO

💡 Usa la función analizar_libro() para explorar:
   analizar_libro("1342", n_recomendaciones=5, m_palabras=10)


In [24]:
analizar_libro("84", n_recomendaciones=5, m_palabras=10)


📊 ANÁLISIS DEL LIBRO: 84

📖 Información:
   Título: 84.txt

📝 Top 10 palabras características:


25/12/06 01:27:17 WARN TaskSetManager: Stage 13 contains a task of very large size (63793 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

    1. elizabeth       (TF-IDF: 129.00)
    2. feelings        (TF-IDF: 24.49)
    3. william         (TF-IDF: 21.19)
    4. misery          (TF-IDF: 19.37)
    5. henry           (TF-IDF: 18.15)
    6. chapter         (TF-IDF: 17.54)
    7. beheld          (TF-IDF: 17.35)
    8. countenance     (TF-IDF: 16.19)
    9. science         (TF-IDF: 15.67)
   10. towards         (TF-IDF: 15.62)

🎯 Top 5 libros similares:
   1. [0.6763] 1342.txt
   2. [0.4039] 2148.txt
   3. [0.3877] 4085.txt
   4. [0.3120] 205.txt
   5. [0.2673] 209.txt


False